Modélisation

In [1]:
#bibliothèques
import statsmodels.api as sm
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss, roc_curve, recall_score, confusion_matrix
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import pearsonr, chi2
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

In [4]:
data = pd.read_csv("data.csv")
data.columns
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50697 entries, 0 to 50696
Data columns (total 31 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Date                                      50697 non-null  object 
 1   Destination Airport                       50697 non-null  object 
 2   Scheduled departure time                  50697 non-null  object 
 3   Actual departure time                     50697 non-null  object 
 4   Wheels-off time                           50697 non-null  object 
 5   Taxi-Out time (Minutes)                   50697 non-null  float64
 6   Delay Carrier (Minutes)                   50697 non-null  float64
 7   Delay Weather (Minutes)                   50697 non-null  float64
 8   Delay National Aviation System (Minutes)  50697 non-null  float64
 9   Delay Security (Minutes)                  50697 non-null  float64
 10  Delay Late Aircraft Arrival (Minut

In [9]:
df = data.drop('time', axis=1)
colu= ["Season","Period_Day","Weekday_Flight","Destination Airport","Date","Scheduled departure time","Actual departure time","Wheels-off time"]
df = df.drop(columns=colu)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50697 entries, 0 to 50696
Data columns (total 22 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Taxi-Out time (Minutes)                   50697 non-null  float64
 1   Delay Carrier (Minutes)                   50697 non-null  float64
 2   Delay Weather (Minutes)                   50697 non-null  float64
 3   Delay National Aviation System (Minutes)  50697 non-null  float64
 4   Delay Security (Minutes)                  50697 non-null  float64
 5   Delay Late Aircraft Arrival (Minutes)     50697 non-null  float64
 6   Retard                                    50697 non-null  float64
 7   Weekday_Flight_encoded                    50697 non-null  float64
 8   Season_encoded                            50697 non-null  float64
 9   Period_Day_encoded                        50697 non-null  float64
 10  Destination_encoded               

In [13]:

# Normalisation des colonnes numériques
scaler = MinMaxScaler()
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

# Division aléatoire en ensembles d'apprentissage et de test
seed = 131
train_set, test_set = train_test_split(df, test_size=0.2, random_state=seed)



In [23]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40557 entries, 49843 to 30579
Data columns (total 22 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Taxi-Out time (Minutes)                   40557 non-null  float64
 1   Delay Carrier (Minutes)                   40557 non-null  float64
 2   Delay Weather (Minutes)                   40557 non-null  float64
 3   Delay National Aviation System (Minutes)  40557 non-null  float64
 4   Delay Security (Minutes)                  40557 non-null  float64
 5   Delay Late Aircraft Arrival (Minutes)     40557 non-null  float64
 6   Retard                                    40557 non-null  float64
 7   Weekday_Flight_encoded                    40557 non-null  float64
 8   Season_encoded                            40557 non-null  float64
 9   Period_Day_encoded                        40557 non-null  float64
 10  Destination_encoded                

In [24]:

# Sous-échantillonnage
rus = RandomUnderSampler(random_state=seed)
train_undersampled, train_undersampled_labels = rus.fit_resample(train_set.drop('Retard', axis=1),
                                                                 train_set['Retard'])
print(type(train_undersampled))
print(train_undersampled.describe())

# Sur-échantillonnage
ros = RandomOverSampler(random_state=seed)
train_oversampled, train_oversampled_labels = ros.fit_resample(train_set.drop('Retard', axis=1),
                                                               train_set['Retard'])


<class 'pandas.core.frame.DataFrame'>
       Taxi-Out time (Minutes)  Delay Carrier (Minutes)  \
count             25844.000000             25844.000000   
mean                  0.137334                 0.004738   
std                   0.072377                 0.025319   
min                   0.000000                 0.000000   
25%                   0.096045                 0.000000   
50%                   0.118644                 0.000000   
75%                   0.152542                 0.000000   
max                   0.926554                 1.000000   

       Delay Weather (Minutes)  Delay National Aviation System (Minutes)  \
count             25844.000000                              25844.000000   
mean                  0.000852                                  0.003953   
std                   0.012844                                  0.015173   
min                   0.000000                                  0.000000   
25%                   0.000000                    

In [28]:


x_train = train_undersampled,
y_train = train_oversampled_labels

print("Dimensions de x_train :", x_train.shape)
print("Dimensions de y_train :", y_train.shape)


Dimensions de x_train : (50697, 22)
Dimensions de y_train : (55270,)


In [30]:
# Modèle de régression logistique
x_train = train_oversampled
y_train = train_oversampled_labels
clf = LogisticRegression()
clf.fit(x_train, y_train)

# Coefficients du modèle
coefficients = clf.coef_[0]
intercept = clf.intercept_[0]
coeff_df = pd.DataFrame({'Variable': x_train.columns, 'Coefficient': coefficients})
coeff_df.loc[len(coeff_df)] = ['Intercept', intercept]
print(coeff_df)

# Calcul des statistiques
n = len(y_train)
p = x_train.shape[1]
X = np.hstack((np.ones((n, 1)), x_train))
y_pred = clf.predict(x_train)
residus = y_train - y_pred
sigma_hat = np.sqrt(np.sum(residus ** 2) / (n - p))
var_cov_beta = np.linalg.inv(X.T @ X) * sigma_hat ** 2
std_err_beta = np.sqrt(np.diag(var_cov_beta))

# Variables significatives
alpha = 0.1
z_score = clf.coef_ / std_err_beta[1:]
p_values = 2 * (1 - chi2.cdf(z_score ** 2, 1))
significant_vars = x_train.columns[np.array(p_values).reshape(-1) < alpha]
print("Variables significatives :", significant_vars)

# Modèle réduit aux variables significatives
significant_coefs = clf.coef_[0][np.isin(x_train.columns, significant_vars)]
significant_model = pd.DataFrame({'Variable': significant_vars, 'Coefficient': significant_coefs})
print("Modèle significatif :\n", significant_model)

# Corrélation de Pearson
pearson_correlation = []
pearson_p_value = []
for var in x_train.columns:
    correlation, p_value = pearsonr(x_train[var], y_train)
    pearson_correlation.append(correlation)
    pearson_p_value.append(p_value)

alpha = 0.05
significant_vars = [var for var, p_value in zip(x_train.columns, pearson_p_value) if p_value < alpha]
print("Variables significatives (avec une erreur alpha de 5%) : \n", significant_vars)


                                    Variable  Coefficient
0                    Taxi-Out time (Minutes)     2.765947
1                    Delay Carrier (Minutes)    25.028333
2                    Delay Weather (Minutes)     5.823789
3   Delay National Aviation System (Minutes)    10.544061
4                   Delay Security (Minutes)     3.915299
5      Delay Late Aircraft Arrival (Minutes)    24.065809
6                     Weekday_Flight_encoded    -0.107880
7                             Season_encoded    -0.206194
8                         Period_Day_encoded    -0.090898
9                        Destination_encoded    -0.278803
10                       temperature_2m (°C)     0.522651
11                  relative_humidity_2m (%)    -0.663656
12                        precipitation (mm)    -0.419742
13                            snow_depth (m)     0.205710
14                   weather_code (wmo code)     0.329572
15                    surface_pressure (hPa)     0.287332
16            

In [33]:
# Évaluation du modèle
def model_evaluation(model, seuil):
    predictions = model.predict_proba(test_set.drop('Retard', axis=1))[:, 1]
    predicted_status = (predictions > seuil).astype(int)
    conf_mat = pd.crosstab(test_set['Retard'], predicted_status)
    TP = conf_mat.iloc[1, 1]
    TN = conf_mat.iloc[0, 0]
    FP = conf_mat.iloc[0, 1]
    FN = conf_mat.iloc[1, 0]
    accuracy = (TP + TN) / len(test_set)
    sensitivity = TP / (FN + TP)
    specificity = TN / (TN + FP)
    return {
        'Confusion Matrix': conf_mat,
        'Classification Score': accuracy,
        'Model Sensitivity': sensitivity,
        'Model Specificity': specificity
    }

# Résultats du modèle
results = model_evaluation(clf, 0.3)
print(results)

{'Confusion Matrix': col_0     0     1
Retard           
0.0     103  6873
1.0       7  3157, 'Classification Score': 0.3214990138067061, 'Model Sensitivity': 0.9977876106194691, 'Model Specificity': 0.014764908256880734}


In [34]:
# Courbes ROC et métriques
def print_results(model):
    seuils = np.arange(0.01, 1.0, 0.01)
    acc_model, sens_model, spec_model, one_minus_spec_model = [], [], [], []
    for i in seuils:
        r = model_evaluation(model, i)
        acc_model.append(r['Classification Score'])
        sens_model.append(r['Model Sensitivity'])
        spec_model.append(r['Model Specificity'])
        one_minus_spec_model.append(1 - r['Model Specificity'])
    result_df = pd.DataFrame({
        'Thresholds': seuils,
        'Accuracy': acc_model,
        'Sensitivity': sens_model,
        'Specificity': spec_model,
        '1-Specificity': one_minus_spec_model
    })
    
    # Courbe ROC
    plt.figure()
    plt.plot(result_df['1-Specificity'], result_df['Sensitivity'])
    plt.xlabel('1-Specificity')
    plt.ylabel('Sensitivity')
    plt.title('ROC Curve')
    plt.show()

    # Courbes des métriques
    plt.figure()
    plt.plot(result_df['Thresholds'], result_df['Accuracy'], label='Accuracy', color='red')
    plt.plot(result_df['Thresholds'], result_df['Sensitivity'], label='Sensitivity', color='green')
    plt.plot(result_df['Thresholds'], result_df['Specificity'], label='Specificity', color='blue')
    plt.xlabel('Threshold')
    plt.ylabel('Score')
    plt.legend()
    plt.title('Model Metrics')
    plt.show()

print_results(clf)


IndexError: index 1 is out of bounds for axis 0 with size 1